In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyroomacoustics as pra
from IPython.display import Audio
from scipy.io import wavfile
import torchaudio
from tqdm import tqdm

In [2]:
data = torchaudio.datasets.LIBRISPEECH(root='.\data_libri', url='train-clean-100', download=False)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\maksim.groshev\AppData\Local\Temp\ipykernel_12264\2569675378.py:1: SyntaxWarning: invalid escape sequence '\d'
  data = torchaudio.datasets.LIBRISPEECH(root='.\data_libri', url='train-clean-100', download=False)


In [3]:
len(data)

28539

In [4]:
def generate_data(audio, fs):
    # характеристики комнаты
    width = np.random.randint(2, 9)  # ширина
    length = np.random.randint(2, 9) # длина
    height = np.random.randint(2, 6) # высота

    rt60 = np.round(np.random.uniform(0.3, 0.7), 2) # временя, которое требуется RIR для затухания на 60 дБ
    e_absorption, max_order = pra.inverse_sabine(rt60, [width, length, height]) # поглощение энергии стенкой
    room = pra.ShoeBox([width, length, height], materials=pra.Material(e_absorption), max_order=max_order, fs=fs)
    
    # характеристики источника
    source_x, source_y, source_z = np.random.randint(0, width), np.random.randint(0, length), np.random.randint(0, height) # координаты источников
    room.add_source([source_x, source_y, source_z], signal=audio, delay=0.0)

    interval = np.round(np.random.uniform(0.05, 1), 2) # интервал микрофонов от 5 см до 1 метра
    count_microphone = 4 #np.random.randint(1, 5) # количество микрофонов
    
    # характеристики микрофонов
    mic_x, mic_y, mic_z = np.random.uniform(0, width), np.random.uniform(0, length), np.random.uniform(0, height) #начальные координаты микрофона
    R = pra.linear_2D_array([mic_x, mic_y], count_microphone, 0, interval) # массив микрофонов (линия)
    mic_locs = np.row_stack((R, np.zeros(count_microphone) + mic_z)) # координаты в пространстве микрофонов

    count_iter = 0
    while not(room.is_inside(mic_locs.T)): # генерируем кординаты микрофонов: пока все мик. не будут в комнате
        
        mic_x, mic_y, mic_z = np.random.uniform(0, width), np.random.uniform(0, length), np.random.uniform(0, height)
        R = pra.linear_2D_array([mic_x, mic_y], count_microphone, 0, interval)
        
        mic_locs = np.row_stack((R, np.zeros(count_microphone)+mic_z))
        count_iter += 1
        if count_iter > 15: # если в течении 15 итераций не удается сегнерировать координаты микрофонов, то изменяем интервал
            interval = np.round(np.random.uniform(0.05, 1), 2)
        
    room.add_microphone_array(mic_locs)
    
    return room, width, length, height

In [5]:
def data_simultate(room, path_save_audio):
    room.simulate()
    room.mic_array.to_wav(
    f"{path_save_audio}.wav"
    )

# train data

In [6]:
# import random

In [7]:
# n = 250

# numbers = list(range(len(data)))

# selected_numbers = random.sample(numbers, k=n)
# print(selected_numbers)

[27218, 26096, 15903, 22103, 10624, 25629, 21540, 21714, 5626, 27034, 5687, 4697, 2636, 16320, 24052, 11294, 24084, 24226, 16324, 17292, 23719, 7433, 11127, 25599, 5561, 16708, 15578, 3462, 11928, 21851, 23698, 19915, 10250, 7372, 22287, 24293, 5660, 18809, 4842, 15142, 23051, 20882, 3710, 23825, 1168, 7102, 14502, 24799, 249, 8199, 9947, 3893, 24800, 28083, 13133, 1746, 26871, 17835, 21442, 7338, 11075, 11837, 27403, 22283, 8229, 12762, 14973, 25500, 22422, 6285, 23319, 1518, 19459, 14187, 6633, 8875, 27115, 19026, 5040, 1758, 10530, 8617, 23093, 5108, 7616, 8266, 6538, 19643, 15672, 4923, 4208, 2075, 9275, 25038, 20803, 14409, 15971, 3045, 13613, 23741, 11557, 25950, 26720, 2652, 12562, 18522, 21396, 19624, 25395, 10159, 5280, 12162, 24960, 5812, 21813, 17635, 28198, 8089, 23679, 6018, 4779, 13170, 25986, 13077, 6741, 15074, 16615, 23713, 9578, 18454, 6823, 1237, 5017, 21365, 21256, 20614, 13351, 27619, 15947, 19067, 10514, 707, 4240, 7223, 24366, 20362, 175, 21083, 2235, 21506, 2369

In [19]:
# for idx in tqdm(selected_numbers):
#     signal = data[idx][0][0].numpy()
#     wavfile.write(f"./beam_data/train/target/{idx}.wav", 16000, signal)

#     r, width, length, height = generate_data(signal, 16000)
#     data_simultate(r, f"./beam_data/train/mic/{idx}" )

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [03:59<00:00,  1.04it/s]


In [20]:
#  valid_set = list(set(numbers) - set(selected_numbers))

# valid data

In [10]:
# n = 50

In [11]:
# valid_samples = random.sample(valid_set, k=n)

In [21]:
# for idx in tqdm(valid_samples):
#     signal = data[idx][0][0].numpy()
#     wavfile.write(f"./beam_data/valid/target/{idx}.wav", 16000, signal)

#     r, width, length, height = generate_data(signal, 16000)
#     data_simultate(r, f"./beam_data/valid/mic/{idx}" )

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:55<00:00,  1.10s/it]


In [22]:
# import os

In [25]:
# len(os.listdir('./beam_data/valid/target'))

50